### Requirements/Libraries


In [1]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON

### Query local Playground dataset

In [4]:
g = Graph()
g.parse("playground.ttl", format="ttl")

print("Loaded '" + str(len(g)) + "' triples.")

print("Query results (Females):")

qres = g.query(
"""SELECT ?thing ?name where {
  ?thing tto:sex "female" .
  ?thing dbp:name ?name .
}""")

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is female with name '%s'" % (str(row.thing),str(row.name)))

Loaded '73' triples.
Query results (Females):
http://example.org/tuto/resource#LunaCat is female with name 'Luna'
http://example.org/tuto/resource#Eve is female with name 'Eve'


### Query local Nobel Prize dataset

In [9]:
g = Graph()
g.parse("../files/nobelprize_kg.nt", format="nt")
  
print("Loaded '" + str(len(g)) + "' triples.")

#Query with prefixes 
nobelprize_query = """
PREFIX nobel: <http://data.nobelprize.org/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?name_laur WHERE { 
?laur rdf:type nobel:Laureate . 
?laur rdfs:label ?name_laur . 
?laur foaf:gender "female" . }
"""

qres = g.query(nobelprize_query)

print("Female laureates (printing 10):")
i=0
for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    i+=1
    print("'%s'" % (str(row.name_laur))) #Same name as in sparql query SELECT variable
    if i==10:
        break

Loaded '85797' triples.
Female laureates (printing 10):
'Malala Yousafzai'
'Youyou Tu'
'Selma Ottilia Lovisa Lagerlöf'
'Ada E. Yonath'
'Elfriede Jelinek'
'Leymah Gbowee'
'Nelly Sachs'
'Wislawa Szymborska'
'Pearl Buck'
'Mairead Corrigan'


### Query remote dataset via SPARQL Enpoint

In [11]:
endpoint_url = "http://dbpedia.org/sparql"
sparqlw = SPARQLWrapper(endpoint_url)
sparqlw.setReturnFormat(JSON)


query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
PREFIX dbo: <http://dbpedia.org/ontology/> 
SELECT DISTINCT ?x WHERE { 
?jd foaf:name \"Johnny Depp\"@en . 
?m dbo:starring ?jd .
?m dbo:starring ?other . 
?other foaf:name ?x . 
FILTER (STR(?x)!=\"Johnny Depp\")
} 
ORDER BY ?x
"""

sparqlw.setQuery(query)

results = sparqlw.query().convert()

#Prints JSON file
#print(results)   

print("Stars co-starring Johnny Depp (printing 10):")
i=0
for result in results["results"]["bindings"]:
    i+=1
    #Prints individual results 
    print(result["x"]["value"])
    if i==10:
        break

Stars co-starring Johnny Depp (printing 10):

Abigail Breslin
Aidan Quinn
Al Pacino
Alain Chabat
Alan Arkin
Albert Tsai
Alfred Molina
Alison Sudol
Amanda Wyss
